In [1]:
cd notebooks/

/home/nicit/deep_learning_course_wfco/notebooks


In [2]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
import torch

import sys
sys.path.append("../src")
import utils

In [3]:
folder_path = "/home/nicit/nordsoen/nordsoen-i-wflo/results/DL/recorders/"

recorders = {}
n_layouts = 100
# load pickle
for random_state in trange(int(n_layouts/100), desc="Loading recorders"):
    with open(folder_path + f"recorder_{random_state}.pkl", "rb") as f:
        recorders[random_state] = pickle.load(f)

Loading recorders:   0%|          | 0/1 [00:00<?, ?it/s]

Loading recorders: 100%|██████████| 1/1 [00:02<00:00,  2.93s/it]


In [4]:
recorders[0][1000].keys()

dict_keys(['yaw', 'x', 'y', 'x_rot', 'y_rot', 'd_x', 'd_y', 'sort_idx', 'ws_eff', 'ti_eff', 'ws', 'aep', 'aep_opt'])

In [5]:
print('x_rot shape: ', recorders[0][1000]['x_rot'].shape)
print('y_rot shape: ', recorders[0][1000]['y_rot'].shape)
print('yaw shape: ', recorders[0][1000]['yaw'].shape)
print('ws_eff shape: ', recorders[0][1000]['ws_eff'].shape)
print('ti_eff shape: ', recorders[0][1000]['ti_eff'].shape)
print('ws shape: ', recorders[0][1000]['ws'].shape)
print('aep shape: ', recorders[0][1000]['aep'])
print('aep_opt shape: ', recorders[0][1000]['aep_opt'].shape)
print('yaw shape: ', recorders[0][1000]['yaw'].shape)

x_rot shape:  (360, 46)
y_rot shape:  (360, 46)
yaw shape:  (46, 360, 11)
ws_eff shape:  (46, 360, 11)
ti_eff shape:  (46, 360, 11)
ws shape:  (11,)
aep shape:  [[0.00437259 0.00587007 0.03076611 ... 0.13747569 0.01684455 0.02444979]
 [0.00434477 0.00583548 0.03062883 ... 0.14218515 0.01743774 0.02528761]
 [0.00431171 0.00579388 0.03045422 ... 0.14669767 0.01801246 0.02614194]
 ...
 [0.00448134 0.00601999 0.0316177  ... 0.17683724 0.02217008 0.03242694]
 [0.00445364 0.00598145 0.03139479 ... 0.16316903 0.02030696 0.02962891]
 [0.00441877 0.00593384 0.03112336 ... 0.15002525 0.01852943 0.02696669]]
aep_opt shape:  (360, 11)
yaw shape:  (46, 360, 11)


In [49]:
n_layouts = 100
rotor_diameter = 284.0  # adjust to your turbine specs
nearest_idx = [0, 1, 2, 3, 4]


# --- Step 1: Load and aggregate all layouts ---
all_x_rot = []
all_y_rot = []
all_ws_eff = []
all_ti_eff = []
all_yaw = []
all_ws = []

for random_state in tqdm(range(int(n_layouts / 100)), desc="Loading recorders"):
    file_path = folder_path + f"recorder_{random_state}.pkl"
    with open(file_path, "rb") as f:
        recorder = pickle.load(f)
    for result in recorder.values():
        all_x_rot.append(result["x_rot"])       # (n_dir, n_turbines)
        all_y_rot.append(result["y_rot"])
        all_ws_eff.append(result["ws_eff"])     # (n_turbines, n_dir, n_ws)
        all_ti_eff.append(result["ti_eff"])
        all_yaw.append(result["yaw"])
        all_ws.append(result["ws"])             # (n_ws,)

# Stack everything
x_rot_all = np.stack(all_x_rot)       # (n_layouts, n_dir, n_turbines)
y_rot_all = np.stack(all_y_rot)
ws_eff_all = np.stack(all_ws_eff)     # (n_layouts, n_turbines, n_dir, n_ws)
ti_eff_all = np.stack(all_ti_eff)
yaw_all = np.stack(all_yaw)
ws_all = np.stack(all_ws)             # (n_layouts, n_ws)

n_layouts_total, n_dir, n_turbines = x_rot_all.shape
n_ws = ws_all.shape[1]
n_cases = n_layouts_total * n_dir * n_ws

# print(f"Aggregated shapes: x_rot_all={x_rot_all.shape}, ws_eff_all={ws_eff_all.shape}")

# --- Step 2: Compute dx, dy for all layouts and directions ---
# Flatten layouts and directions for _process_layout
x_rot_flat = x_rot_all.reshape(-1, n_turbines)  # (n_layouts * n_dir, n_turbines)
y_rot_flat = y_rot_all.reshape(-1, n_turbines)

dx_all, dy_all = utils._process_layout(
    turbine_x=x_rot_flat,
    turbine_y=y_rot_flat,
    rotor_diameter=rotor_diameter,
    nearest_idx=nearest_idx,
    normalize=True,
    spread=.1
)  # shape: (len(nearest_idx), n_turbines, n_layouts * n_dir)

# Repeat across wind speeds
dx_flat = np.repeat(dx_all, n_ws, axis=2).reshape(len(nearest_idx), n_turbines, n_cases)
dy_flat = np.repeat(dy_all, n_ws, axis=2).reshape(len(nearest_idx), n_turbines, n_cases)

# --- Step 3: Flatten dynamic inputs ---
ws_eff_flat = ws_eff_all.reshape(n_layouts_total, n_turbines, n_dir * n_ws)
ws_eff_flat = ws_eff_flat.transpose(1, 0, 2).reshape(n_turbines, n_cases)

ti_eff_flat = ti_eff_all.reshape(n_layouts_total, n_turbines, n_dir * n_ws)
ti_eff_flat = ti_eff_flat.transpose(1, 0, 2).reshape(n_turbines, n_cases)

yaw_flat = yaw_all.reshape(n_layouts_total, n_turbines, n_dir * n_ws)
yaw_flat = yaw_flat.transpose(1, 0, 2).reshape(n_turbines, n_cases)

# Repeat ws for all layouts and directions
# ws_rep = np.tile(ws_all.reshape(-1), n_dir)  # (n_cases,)
ws_rep = np.tile(ws_all.reshape(n_layouts_total, n_ws), (1, n_dir)).reshape(n_cases)

# --- Step 4: Build feature matrix in one shot ---
dx_part = dx_flat.transpose(1, 2, 0)  # (n_turbines, n_cases, 5)
dy_part = dy_flat.transpose(1, 2, 0)

# modify this so that it is -1 when it is not waked (or waked), instaed of zero!
waked_part = np.where(dx_part != np.inf, 1.0, -1.0)

# Replace inf with nan for dx/dy
dx_part[dx_part == np.inf] = np.nan

# when there is nan in the corresponding dx, set dy to nan as well
dy_part[dy_part == np.inf] = np.nan

# Ensure dy is NaN wherever dx is NaN
dy_part[np.isnan(dx_part)] = np.nan

# adjust so that whenever the ws_eff is >= 12, the yaw should be zero
yaw_flat = np.where(ws_eff_flat >= 12.0, 0.0, yaw_flat)

# same whenever ws_eff is < 4
yaw_flat = np.where(ws_eff_flat < 4.0, 0.0, yaw_flat)

# Static features: dx, dy, waked flags
static_features = np.concatenate([dx_part, dy_part, 
                                waked_part
                                  ], axis=2)  # (n_turbines, n_cases, 15)

# merge ws and ws_eff into one feature (the difference)
dynamic_features = np.stack([
    ws_eff_flat,
    ti_eff_flat,
    np.tile(ws_rep, (n_turbines, 1))
], axis=2)

# Combine all features
X = np.concatenate([static_features, dynamic_features], axis=2)  # (n_turbines, n_cases, 18)
X = X.reshape(-1, X.shape[2])  # (n_turbines * n_cases, 18)
Y = yaw_flat.reshape(-1)

# Convert to torch tensors
X_torch = torch.tensor(X, dtype=torch.float32)
Y_torch = torch.tensor(Y, dtype=torch.float32)

print(f"Final dataset shapes: X = {X_torch.shape}, Y = {Y_torch.shape}")

# Save dataset
torch.save((X_torch, Y_torch), f"../data/5wt_dataset_{n_layouts}.pt")
print(f"Dataset saved to 5wt_dataset_{n_layouts}.pt")


Loading recorders: 100%|██████████| 1/1 [00:01<00:00,  1.17s/it]


Final dataset shapes: X = torch.Size([18216000, 18]), Y = torch.Size([18216000])
Dataset saved to 5wt_dataset_100.pt


In [50]:
# check nans in X
print('Number of NaNs in X: ', torch.isnan(X_torch).sum().item())
print('Shape of X: ', X_torch.shape)
print("Number of total data points in X", X_torch.shape[0]*X_torch.shape[1])
print("Percentage of NaNs in X", round(torch.isnan(X_torch).sum().item()/(X_torch.shape[0]*X_torch.shape[1])*100, 2), "%")

Number of NaNs in X:  125842046
Shape of X:  torch.Size([18216000, 18])
Number of total data points in X 327888000
Percentage of NaNs in X 38.38 %


In [51]:
# i 10 is the weird stuff
i = 15840
print(X_torch[i])
print(Y_torch[i])

tensor([ 8.3937,     nan,     nan,     nan,     nan, -1.7280,     nan,     nan,
            nan,     nan,  1.0000, -1.0000, -1.0000, -1.0000, -1.0000,  3.0525,
         0.0600,  3.0525])
tensor(0.)


In [47]:
# get index where last value is < 3.2
indices = (X_torch[:, -1] < 3.2).nonzero(as_tuple=True)[0]
print(f"Number of samples with wind speed < 3.2 m/s: {len(indices)}")
indices[0]

Number of samples with wind speed < 3.2 m/s: 314640


tensor(15840)